In [2]:
import optuna
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import xgboost as xgb
from sklearn.metrics import accuracy_score

import warnings
warnings.filterwarnings("ignore")

c:\Users\Administrator\Desktop\FinetuningOptuna\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
df = pd.read_csv("heart.csv")
df = pd.get_dummies(df)
X = df.drop("HeartDisease", axis=1).copy()
y = df["HeartDisease"].copy()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=369)


In [6]:
model = xgb.XGBClassifier(device = "cuda")
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
accuracy_score(y_test, y_pred)


0.8768115942028986

In [7]:
def objective(trial):
    hiper = {
        'n_estimators': trial.suggest_int('n_estimators', 100, 1000),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3, log=True),
        'subsample': trial.suggest_float('subsample', 0.6, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0),
        'reg_alpha': trial.suggest_float('reg_alpha', 1e-8, 1.0, log=True),
        'reg_lambda': trial.suggest_float('reg_lambda', 1e-8, 1.0, log=True),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
        'gamma': trial.suggest_float('gamma', 0, 0.5),
        'device': 'cuda',  # SI TIENES GPU
    }
    model = xgb.XGBClassifier(**hiper)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    return acc
study  = optuna.create_study(direction="maximize")
study.optimize(objective, n_jobs=-1, n_trials=1000)

[I 2025-08-22 21:15:50,170] A new study created in memory with name: no-name-96e5be8a-91d3-43c1-affa-4760f59d8d9c
[I 2025-08-22 21:15:57,200] Trial 11 finished with value: 0.9021739130434783 and parameters: {'n_estimators': 247, 'max_depth': 3, 'learning_rate': 0.011766437101264017, 'subsample': 0.7625074998833445, 'colsample_bytree': 0.6997696144896075, 'reg_alpha': 0.7005883776408239, 'reg_lambda': 1.1140557622928843e-05, 'min_child_weight': 5, 'gamma': 0.29284045899994854}. Best is trial 11 with value: 0.9021739130434783.
[I 2025-08-22 21:15:59,190] Trial 2 finished with value: 0.894927536231884 and parameters: {'n_estimators': 314, 'max_depth': 3, 'learning_rate': 0.020927006705115075, 'subsample': 0.996768283085641, 'colsample_bytree': 0.8378096137916047, 'reg_alpha': 1.9597513472749766e-06, 'reg_lambda': 0.0024208115186301195, 'min_child_weight': 9, 'gamma': 0.39079821873736686}. Best is trial 11 with value: 0.9021739130434783.
[I 2025-08-22 21:15:59,202] Trial 0 finished with va

In [15]:
model = xgb.XGBClassifier(**study.best_params)
print(study.best_params)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
acc  = accuracy_score(y_test, y_pred)
print(acc)

{'n_estimators': 231, 'max_depth': 5, 'learning_rate': 0.01273679665478278, 'subsample': 0.7154399840087362, 'colsample_bytree': 0.7779524962793323, 'reg_alpha': 1.7065084306733312e-06, 'reg_lambda': 0.10212707994391979, 'min_child_weight': 5, 'gamma': 0.3054236634578809}
0.9130434782608695
